In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from backend.models import Video
from backend.model_helpers import query_or, query_and

In [2]:
Video.objects.all().count()

3082

In [3]:
qs = Video.objects.all()

In [4]:
def annotate_with_n_reports(qs, username=None, prefix=''):
    """Add n_reports field.
    
    By-default, Video queryset is assumed.
    This can be changed via the prefix argument.
    
    If username is not None, only reports by this person
    are counted.
    
    Args:
        qs: Input queryset (Video by-default)
        username: if not None, filter only reports by this person
        prefix: prepend this to field names
    Returns:
        qs with n_reports integer field
    """
    filter_ = None
    
    if username is not None:
        filter_ = Q(**{prefix + 'videoreports__user__user__username': username})
    
    qs = qs.annotate(n_reports=Count(prefix + 'videoreports',
                                     filter=filter_,
                                     distinct=True))
    
    return qs

In [5]:
annotate_with_n_reports(qs).filter(n_reports__gt=0).values('video_id', 'n_reports')

<QuerySet [{'video_id': '9Ay4u7OYOhA', 'n_reports': 1}, {'video_id': 'PMOf0uD1xaglist', 'n_reports': 1}, {'video_id': 'rlDtOnhHAe8', 'n_reports': 2}, {'video_id': 'DCvFDegqPnw', 'n_reports': 1}]>

In [14]:
def annotate_with_n_ratings(qs, only_certified=True, username=None, prefix=''):
    """Add rating_n_ratings and rating_n_experts fields.
    
    By-default, Video queryset is assumed.
    This can be changed via the prefix argument.
    
    If username is not None, only ExpertRating objects by this
    person are counted.
    
    Args:
        qs: Input queryset (Video by-default)
        only_certified: only count ratings from certified people
        username: if not None, only count ratings by this person
        prefix: prepend this to field names
        
    Returns:
        qs with rating_n_ratings [total number of ExpertRating objects],
            rating_n_experts [total number of people in ExpertRating objects]
    """
    
    for vid in '12':
        # ONLY CERTIFIED EXPERTS filter
        if only_certified:
            # _is_certified_{vid} represents if a user with the corresponding expert rating is certified
            qs = UserInformation._annotate_is_certified(
                qs, prefix=prefix + f'expertrating_video_{vid}__user__user__userinformation__',
                output_field=f'_is_certified_{vid}')
            filter_cert = Q(**{f'_is_certified_{vid}': 1})
        else:
            filter_cert = None

        # EXPERT USERNAME filter
        if username is None:
            filter_username = None
        else:
            filter_username = Q(**{prefix + f'expertrating_video_{vid}__user__user__username': username})
            
        # total filter
        filter_ = query_and([filter_cert, filter_username])
            
        # counting the number of ratings
        qs = qs.annotate(**{f'rating_n_ratings_{vid}':
                            Count(prefix + f'expertrating_video_{vid}',
                                  filter=filter_, distinct=True)})
        
#         qs = qs.annotate(**{f'rating_n_experts_{vid}':
#                             Count(prefix + f'expertrating_video_{vid}',
#                                   filter=filter_, distinct=True)})
    
    return qs

In [15]:
annotate_with_n_ratings(qs)

ProgrammingError: aggregate functions are not allowed in FILTER
LINE 1: ...kend_expertrating"."id") FILTER (WHERE CASE WHEN (COUNT("bac...
                                                             ^


<function backend.models.UserInformation._annotate_is_certified(queryset, prefix='')>